In [8]:
#for autoEncDec
import pyperclip

py_cmd = ['py','python','python3'][2]
image_path = './data/images/'
img_count = -1
epochs = 300
batch_size = 8
loss = 'msle'
gpu_num = 3
cmd =   f"""
{py_cmd} autoEncDec.py --image_path {image_path} 
--img_count {img_count} --epochs {epochs} 
--batch_size {batch_size} --loss {loss} 
--gpu_num {gpu_num}
"""
cmd = cmd.replace('\n','')

pyperclip.copy(cmd)
cmd

'python3 autoEncDec.py --image_path ./data/images/ --img_count -1 --epochs 300 --batch_size 8 --loss msle --gpu_num 3'

In [10]:
#for test_model
import pyperclip

py_cmd = ['py','python','python3'][2]

seq_len = 24
n_seq = 6
hidden_dim = 6
gamma = 1

noise_dim = 32
dim = 128
batch_size = 8

log_step = 100
lr = 5e-4
gpu_num = 3

image_path = './data/images/'
img_count = -1

cmd =   f"""
{py_cmd} autoEncDec.py --seq_len {seq_len} --n_seq {n_seq} 
--hidden_dim {hidden_dim} --gamma {gamma} 
--noise_dim {noise_dim} --dim {dim} --batch_size {batch_size} 
--log_step {log_step} --lr {lr} --gpu_num {gpu_num} 
--image_path {image_path} --img_count {img_count}
"""
cmd = cmd.replace('\n','')

pyperclip.copy(cmd)
cmd

'python3 autoEncDec.py --seq_len 24 --n_seq 6 --hidden_dim 6 --gamma 1 --noise_dim 32 --dim 128 --batch_size 8 --log_step 100 --lr 0.0005 --gpu_num 3 --image_path ./data/images/ --img_count -1'